In [1]:
# https://towardsdatascience.com/basic-binary-sentiment-analysis-using-nltk-c94ba17ae386
import nltk
import random
from nltk.classify.scikitlearn import SklearnClassifier
import pickle
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from nltk.classify import ClassifierI
from statistics import mode
from nltk.tokenize import word_tokenize
import re
import pandas as pd

In [2]:
df = pd.read_csv('kp_date.csv')
df.head()

,Unnamed: 0,title,score,id,url,comms_num,created,body,topcomment,comment2,date
0,0,Kristaps Porzingis boasting jacked arms ahead ...,15,cqf02y,https://nypost.com/2019/08/14/kristaps-porzing...,34,1.565842e+09,NaN,Wow he actually put a lot of size on,I thought shit on NYK day was Tuesday?,2019-08-15 06:59:58
1,1,"Who is gonna get booed the most? Kyrie,AD,Porz...",12,clrtrd,https://www.reddit.com/r/nba/comments/clrtrd/w...,62,1.564919e+09,Who’s most likely to get booed when they retur...,porzingis no question,Kyrie,2019-08-04 14:35:23
2,2,Did Dallas truly beat the Knicks in the KP trade?,0,cqgz75,https://www.reddit.com/r/nba/comments/cqgz75/d...,22,1.565850e+09,Last season in the league:\n\nJulius Randle (2...,Randle is completely irrelevant to the trade,Why have i seen “julius is basically just as g...,2019-08-15 09:26:12
3,3,Kristaps Porzingis Mavericks Debut Highlights,41,dfqybt,https://youtu.be/t34IWAI6IiA,73,1.570700e+09,NaN,Anyone citing his 18/7 needs to realize he onl...,career high 7 rebounds,2019-10-10 12:38:22
4,4,[Highlight] Porzingis with his first Mavs bucket,2088,dfp6y0,https://streamable.com/f6mic,292,1.570692e+09,NaN,"**Replays/alternate angles**\n\n*I am a bot, a...",Why does he look 10 years older,2019-10-10 10:15:39


In [3]:
topcomments = list(df['topcomment'])
topcomments[:5]

['Wow he actually put a lot of size on',
 'porzingis no question',
 'Randle is completely irrelevant to the trade',
 'Anyone citing his 18/7 needs to realize he only played 19 minutes',
 '**Replays/alternate angles**\n\n*I am a bot, and this action was performed automatically. Please [contact the moderators of this subreddit](/message/compose/?to=/r/nba) if you have any questions or concerns.*']

In [4]:
topcomments[-10:]

['Smart decision but you can understand why Porzingis would be upset ',
 'The lakers wouldn’t have been willing to take on Lee and THJ',
 'Teams talking about adding a third piece to their two franchise players and I’m out here excited for my 22 YO PG averaging 13 PPG',
 "Might not be something Knicks fans want to hear but they overrate their players. They love Knick players almost unconditionally even if they have faults. some of that of course is because the front office has been the core of the chaos 99% of the time but you should call a spade a spade; most of the allure of Porzingis is the potential, not what he is right now. Not saying that he's a bum but losing him isn't the end of the world and it might turn out to be a good thing in the long run.",
 "I hope the Knicks don't trade Rebecca.",
 'Jokic',
 "This was always the plan, please don't overreact. ",
 '### **Replays/alternate angles**\n                \n\n\n*I am a bot, and this action was performed automatically. Please [c

In [6]:
# shortest comments
shortest = sorted(topcomments, key = lambda el: len(el))
shortest[:5]

['lol', 'Lmao', 'Jesus', 'Yeees', 'Fuck.']

In [7]:
longest = shortest[::-1]
longest[:5]

['News about KP Trade:\n\n[[Stein] The Knicks would receive Dennis Smith Jr. and the expiring contracts of DeAndre Jordan and Wes Matthews in the proposed trade, according to league sources](https://twitter.com/TheSteinLine/status/1091073144851480576)\n\n[[Stein] The Mavericks have agreed to take on the contracts of Tim Hardaway Jr. and Courtney Lee to faciliate a Porzingis trade and are working to complete the transaction today, league sources say](https://twitter.com/TheSteinLine/status/1091073004988235776)\n\n[[Stein] The Knicks would also receive future first-round draft compensation from the Mavericks and, by shedding the contracts of Hardaway and Lee, create the salary-cap space this summer to pursue two maximum contract free agents](https://twitter.com/TheSteinLine/status/1091073212887244801?s=19)\n\n[[Stein] The Knicks would also receive future first-round draft compensation from the Mavericks and, by shedding the contracts of Hardaway and Lee, create the salary-cap space this 

In [8]:
longest[3]

"Might not be something Knicks fans want to hear but they overrate their players. They love Knick players almost unconditionally even if they have faults. some of that of course is because the front office has been the core of the chaos 99% of the time but you should call a spade a spade; most of the allure of Porzingis is the potential, not what he is right now. Not saying that he's a bum but losing him isn't the end of the world and it might turn out to be a good thing in the long run."

In [10]:
comment = nltk.Text(longest[3])

In [15]:
comment.

AttributeError: 'Text' object has no attribute 'text'

In [17]:
# https://www.nltk.org/book/ch05.html
# Using a Tagger
tagged = nltk.pos_tag(nltk.word_tokenize(longest[3]))
tagged

[('Might', 'NNP'),
 ('not', 'RB'),
 ('be', 'VB'),
 ('something', 'NN'),
 ('Knicks', 'NNS'),
 ('fans', 'NNS'),
 ('want', 'VBP'),
 ('to', 'TO'),
 ('hear', 'VB'),
 ('but', 'CC'),
 ('they', 'PRP'),
 ('overrate', 'VBP'),
 ('their', 'PRP$'),
 ('players', 'NNS'),
 ('.', '.'),
 ('They', 'PRP'),
 ('love', 'VBP'),
 ('Knick', 'NNP'),
 ('players', 'NNS'),
 ('almost', 'RB'),
 ('unconditionally', 'RB'),
 ('even', 'RB'),
 ('if', 'IN'),
 ('they', 'PRP'),
 ('have', 'VBP'),
 ('faults', 'NNS'),
 ('.', '.'),
 ('some', 'DT'),
 ('of', 'IN'),
 ('that', 'DT'),
 ('of', 'IN'),
 ('course', 'NN'),
 ('is', 'VBZ'),
 ('because', 'IN'),
 ('the', 'DT'),
 ('front', 'NN'),
 ('office', 'NN'),
 ('has', 'VBZ'),
 ('been', 'VBN'),
 ('the', 'DT'),
 ('core', 'NN'),
 ('of', 'IN'),
 ('the', 'DT'),
 ('chaos', 'NN'),
 ('99', 'CD'),
 ('%', 'NN'),
 ('of', 'IN'),
 ('the', 'DT'),
 ('time', 'NN'),
 ('but', 'CC'),
 ('you', 'PRP'),
 ('should', 'MD'),
 ('call', 'VB'),
 ('a', 'DT'),
 ('spade', 'NN'),
 ('a', 'DT'),
 ('spade', 'NN'),
 (';', 

In [18]:
nltk.help.upenn_tagset()

$: dollar
    $ -$ --$ A$ C$ HK$ M$ NZ$ S$ U.S.$ US$
'': closing quotation mark
    ' ''
(: opening parenthesis
    ( [ {
): closing parenthesis
    ) ] }
,: comma
    ,
--: dash
    --
.: sentence terminator
    . ! ?
:: colon or ellipsis
    : ; ...
CC: conjunction, coordinating
    & 'n and both but either et for less minus neither nor or plus so
    therefore times v. versus vs. whether yet
CD: numeral, cardinal
    mid-1890 nine-thirty forty-two one-tenth ten million 0.5 one forty-
    seven 1987 twenty '79 zero two 78-degrees eighty-four IX '60s .025
    fifteen 271,124 dozen quintillion DM2,000 ...
DT: determiner
    all an another any both del each either every half la many much nary
    neither no some such that the them these this those
EX: existential there
    there
FW: foreign word
    gemeinschaft hund ich jeux habeas Haementeria Herr K'ang-si vous
    lutihaw alai je jour objets salutaris fille quibusdam pas trop Monte
    terram fiche oui corporis ...
IN: preposition or

In [19]:
# we filter for adverbs and adjectives 
adj = [el[0] for el in tagged if 'JJ' in el[1] or 'RB' in el[1]]
adj

['not',
 'almost',
 'unconditionally',
 'even',
 'most',
 'potential',
 'not',
 'right',
 'now',
 'Not',
 "n't",
 'good',
 'long']

In [34]:
from nltk.corpus import stopwords
def createFeatures(words):
    useful_words = [word for word in words if word not in stopwords.words("english")]
    tagged = nltk.pos_tag(useful_words)
    # filter for adjective
    adj = [el[0] for el in tagged if 'JJ' in el[1] or 'RB' in el[1]]
    return dict([(word, True) for word in adj])

In [35]:
adict = createFeatures(adj)
adict

{'almost': True,
 'unconditionally': True,
 'even': True,
 'potential': True,
 'Not': True,
 "n't": True,
 'good': True,
 'long': True}

In [27]:
from nltk.corpus import movie_reviews


In [42]:
pos_reviews = []
for fileid in movie_reviews.fileids('pos')[30:60]:
    words = movie_reviews.words(fileid)
    pos_reviews.append((createFeatures(words), "positive"))
    
#print(pos_reviews[0])    
print(len(pos_reviews))
 
1000

30


1000

In [37]:
pos_reviews[0]

({'comic': True,
  'plenty': True,
  'ghost': True,
  'never': True,
  'really': True,
  'eddie': True,
  'whole': True,
  'new': True,
  'moore': True,
  'thoroughly': True,
  'subject': True,
  'little': True,
  'odd': True,
  '"': True,
  'long': True,
  'nearly': True,
  'consist': True,
  'dismiss': True,
  'past': True,
  'albert': True,
  'almost': True,
  'ludicrous': True,
  'well': True,
  'better': True,
  'direct': True,
  'violent': True,
  'ghetto': True,
  'whitechapel': True,
  'east': True,
  'filthy': True,
  'sooty': True,
  'nervous': True,
  'mysterious': True,
  'surgical': True,
  'first': True,
  'stiff': True,
  'robbie': True,
  'johnny': True,
  'widower': True,
  'prophetic': True,
  'unsuccessfully': True,
  'quell': True,
  'copious': True,
  'absinthe': True,
  'unfortunate': True,
  'mary': True,
  'kelly': True,
  'horribly': True,
  'gruesome': True,
  'even': True,
  'briefed': True,
  'unique': True,
  'interesting': True,
  'bother': True,
  'vertic

In [40]:
from nltk.classify import NaiveBayesClassifier
classifier = NaiveBayesClassifier.train(pos_reviews)

In [43]:
accuracy = nltk.classify.util.accuracy(classifier, pos_reviews)
print(accuracy * 100)

100.0


In [ ]:
# TODO run the classifier on a much larger set than 30! andmix both positive and negative reviews!!
# ideally at least 10k each of positive and negative


In [51]:
prob =  classifier.prob_classify(adict)